In [2]:
import pandas as pd
import numpy as np
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from darts import TimeSeries
from darts.models import Prophet as DartsProphet, XGBModel, RandomForest, ExponentialSmoothing
from darts.metrics import mape
from sklearn.model_selection import ParameterGrid
import warnings

warnings.filterwarnings('ignore')

# Load the data
df = pd.read_csv('100_SKUS.csv')

# Convert date columns to datetime
date_columns = [col for col in df.columns if col != 'material']
for col in date_columns:
    df[col] = pd.to_datetime(col)

# Function to prepare data for a specific SKU
def prepare_data(sku):
    sku_data = df[df['material'] == sku].melt(id_vars=['material'], var_name='date', value_name='value')
    sku_data['date'] = pd.to_datetime(sku_data['date'])
    sku_data = sku_data.sort_values('date')
    sku_data['value'] = pd.to_numeric(sku_data['value'], errors='coerce')
    sku_data = sku_data.dropna()
    return sku_data

# Function to create Darts TimeSeries
def create_darts_series(data):
    return TimeSeries.from_dataframe(data, 'date', 'value')

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import warnings

warnings.simplefilter('ignore', ConvergenceWarning)

def evaluate_sarima_model(data, order, seasonal_order):
    try:
        model = SARIMAX(data['value'], order=order, seasonal_order=seasonal_order,
                        enforce_stationarity=False, enforce_invertibility=False)
        results = model.fit(disp=False, maxiter=200)
        return results.aic
    except:
        return float('inf')

def optimize_sarima(data, s=12):
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], s) for x in list(itertools.product(p, d, q))]
    
    best_aic = float('inf')
    best_params = None
    best_seasonal_params = None
    
    for param in pdq:
        for seasonal_param in seasonal_pdq:
            aic = evaluate_sarima_model(data, param, seasonal_param)
            if aic < best_aic:
                best_aic = aic
                best_params = param
                best_seasonal_params = seasonal_param
    
    return best_params, best_seasonal_params

def sarima_forecast(data, steps):
    best_params, best_seasonal_params = optimize_sarima(data)
    try:
        model = SARIMAX(data['value'], order=best_params, seasonal_order=best_seasonal_params,
                        enforce_stationarity=False, enforce_invertibility=False)
        results = model.fit(disp=False, maxiter=200)
        forecast = results.forecast(steps)
        return forecast
    except:
        print(f"SARIMA forecast failed. Returning zeros.")
        return np.zeros(steps)
# Optimize Prophet parameters
def optimize_prophet(data):
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    }

    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []

    for params in all_params:
        m = Prophet(**params).fit(data)
        df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days')
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    best_params = all_params[np.argmin(rmses)]
    return best_params

# Prophet model
def prophet_forecast(data, steps):
    prophet_data = data.rename(columns={'date': 'ds', 'value': 'y'})
    best_params = optimize_prophet(prophet_data)
    model = Prophet(**best_params)
    model.fit(prophet_data)
    future = model.make_future_dataframe(periods=steps, freq='MS')
    forecast = model.predict(future)
    return forecast['yhat'].iloc[-steps:]

# Optimize XGBoost parameters
def optimize_xgb(series):
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 4],
        'learning_rate': [0.01, 0.1],
    }
    
    best_mape = float('inf')
    best_params = None
    
    for params in ParameterGrid(param_grid):
        model = XGBModel(**params, lags=12)
        mape_score = model.historical_forecasts(
            series,
            start=0.6,
            forecast_horizon=1,
            stride=1,
            retrain=False,
            verbose=False
        ).mean()
        
        if mape_score < best_mape:
            best_mape = mape_score
            best_params = params
    
    return best_params

# Darts models
def darts_forecast(series, steps):
    xgb_params = optimize_xgb(series)
    models = [
        DartsProphet(),
        XGBModel(**xgb_params),
        RandomForest(),
        ExponentialSmoothing()
    ]
    
    forecasts = []
    for model in models:
        model.fit(series)
        forecast = model.forecast(steps)
        forecasts.append(forecast)
    
    return forecasts

# Ensemble forecast
def ensemble_forecast(forecasts):
    return np.mean([f.values() for f in forecasts], axis=0)

# Main forecasting function
def forecast_sku(sku, steps=2):
    data = prepare_data(sku)
    series = create_darts_series(data)
    
    # SARIMA forecast
    sarima_pred = sarima_forecast(data, steps)
    
    # Prophet forecast
    prophet_pred = prophet_forecast(data, steps)
    
    # Darts forecasts
    darts_preds = darts_forecast(series, steps)
    
    # Ensemble
    ensemble_pred = ensemble_forecast(darts_preds)
    
    # Combine all predictions
    all_preds = np.vstack([sarima_pred, prophet_pred, ensemble_pred])
    final_pred = np.mean(all_preds, axis=0)
    
    return final_pred

# Compare with February 2023
def compare_with_feb_2023(sku, prediction):
    actual_feb_2023 = df[df['material'] == sku]['2/1/2023'].values[0]
    print(f"SKU: {sku}")
    print(f"Actual February 2023: {actual_feb_2023}")
    print(f"Predicted February 2024: {prediction[1]:.2f}")
    print(f"Year-over-Year Change: {(prediction[1] - actual_feb_2023) / actual_feb_2023 * 100:.2f}%")
    print()

# Forecast for multiple SKUs
skus_to_forecast = ['sku835', 'sku858', 'sku837', 'sku859', 'sku863']

for sku in skus_to_forecast:
    prediction = forecast_sku(sku)
    compare_with_feb_2023(sku, prediction)

11:49:57 - cmdstanpy - INFO - Chain [1] start processing
11:50:04 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/4 [00:00<?, ?it/s]

11:50:04 - cmdstanpy - INFO - Chain [1] start processing
11:50:04 - cmdstanpy - INFO - Chain [1] done processing
11:50:04 - cmdstanpy - INFO - Chain [1] start processing
11:50:04 - cmdstanpy - INFO - Chain [1] done processing
11:50:04 - cmdstanpy - INFO - Chain [1] start processing
11:50:05 - cmdstanpy - INFO - Chain [1] done processing
11:50:05 - cmdstanpy - INFO - Chain [1] start processing
11:50:05 - cmdstanpy - INFO - Chain [1] done processing
11:50:05 - cmdstanpy - INFO - Chain [1] start processing
11:50:11 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/4 [00:00<?, ?it/s]

11:50:11 - cmdstanpy - INFO - Chain [1] start processing
11:50:11 - cmdstanpy - INFO - Chain [1] done processing
11:50:11 - cmdstanpy - INFO - Chain [1] start processing
11:50:11 - cmdstanpy - INFO - Chain [1] done processing
11:50:11 - cmdstanpy - INFO - Chain [1] start processing
11:50:12 - cmdstanpy - INFO - Chain [1] done processing
11:50:12 - cmdstanpy - INFO - Chain [1] start processing
11:50:13 - cmdstanpy - INFO - Chain [1] done processing
11:50:13 - cmdstanpy - INFO - Chain [1] start processing
11:50:21 - cmdstanpy - INFO - Chain [1] done processing


...

  0%|          | 0/4 [00:00<?, ?it/s]

11:59:00 - cmdstanpy - INFO - Chain [1] start processing
11:59:08 - cmdstanpy - INFO - Chain [1] done processing
11:59:08 - cmdstanpy - INFO - Chain [1] start processing
11:59:14 - cmdstanpy - INFO - Chain [1] done processing
11:59:14 - cmdstanpy - INFO - Chain [1] start processing
11:59:26 - cmdstanpy - INFO - Chain [1] done processing
11:59:26 - cmdstanpy - INFO - Chain [1] start processing
11:59:40 - cmdstanpy - INFO - Chain [1] done processing
11:59:40 - cmdstanpy - INFO - Chain [1] start processing
11:59:48 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/4 [00:00<?, ?it/s]

11:59:48 - cmdstanpy - INFO - Chain [1] start processing
11:59:55 - cmdstanpy - INFO - Chain [1] done processing
11:59:55 - cmdstanpy - INFO - Chain [1] start processing
12:00:07 - cmdstanpy - INFO - Chain [1] done processing
12:00:07 - cmdstanpy - INFO - Chain [1] start processing
12:00:19 - cmdstanpy - INFO - Chain [1] done processing
12:00:19 - cmdstanpy - INFO - Chain [1] start processing
12:00:33 - cmdstanpy - INFO - Chain [1] done processing
12:00:33 - cmdstanpy - INFO - Chain [1] start processing
12:00:39 - cmdstanpy - INFO - Chain [1] done processing
ValueError: The model has not been fitted yet, and `retrain` is ``False``. Either call `fit()` before `historical_forecasts()`, or set `retrain` to something different than ``False``.


ValueError: The model has not been fitted yet, and `retrain` is ``False``. Either call `fit()` before `historical_forecasts()`, or set `retrain` to something different than ``False``.